In [ ]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()
main_db = FAISS.load_local("../DB/db_demo", embeddings, allow_dangerous_deserialization=True)
retriever = main_db.as_retriever(search_kwargs={"k": 10})
llm = Ollama(model="llama2", base_url="http://localhost:11434")

In [ ]:
template = """
Your task is to analyze the reviews of books sold online.
Get the book name and author name from the context provided. 
Answer the question based only on the following context:
{context}"

Don't respond with duplicate data.
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm.invoke
    | StrOutputParser()
)

In [ ]:
chain.invoke( """ what is the avg rating of the book "Number the Stars" """)

In [ ]:
chain.invoke( """ Which are the books published by HarperFlamingo Canada """)